In [3]:
import pandas as pd 

data = pd.read_csv("pvq21CENTRED.csv", sep=',')
lib_data = data.copy()
con_data = data.copy()

In [4]:
def normalise_lib (value):

        if value < 3:
            return 1 #liberal
        else:
            return 0 #moderate

def normalise_con (value):
        
        if value > 6:
            return 1 #conservative
        else:
            return 0 #moderate

## Liberal Model:

In [5]:
lib_data['lrscale'] = lib_data['lrscale'].apply(normalise_lib)

In [6]:
lib_data['lrscale'].head(30)

0     1
1     1
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    1
26    0
27    0
28    0
29    0
Name: lrscale, dtype: int64

In [7]:
import csv, numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
#TEMP 
from sklearn.linear_model import LinearRegression, Lasso

In [8]:
x_df_lib = lib_data.iloc[:, 2:12].copy()
y_df_lib = lib_data.iloc[:,1].copy()

In [9]:
y_df_lib.value_counts(normalize=True)

0    0.884818
1    0.115182
Name: lrscale, dtype: float64

In [15]:
X_lib = x_df_lib.to_numpy()
y_lib = y_df_lib.to_numpy()

In [16]:
X_train_lib, X_test_lib, y_train_lib, y_test_lib = train_test_split(X_lib, y_lib, test_size=0.20, random_state=10)

In [17]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
y_train_lib = y_train_lib.reshape(-1, 1)
y_test_lib = y_test_lib.reshape(-1, 1)
y_train_onehot = enc.fit_transform(y_train_lib)
y_test_onehot = enc.fit_transform(y_test_lib)


In [32]:
import numpy as np
print(y_train_onehot)
rounded_labels=np.argmax(y_train_onehot, axis=1)
rounded_labels[0]


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


0

In [56]:
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train_lib, y_train_onehot, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1).mean()
# force scores to be positive|
# scores = np.absolute(scores)
# print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
scores

-0.2034459251647572

In [59]:
from sklearn.metrics import accuracy_score

model.fit(X_train_lib, y_train_lib)
lasso_predicted = model.predict(X_test_lib)
print(lasso_predicted)
accuracy_score(y_test_onehot, lasso_predicted)

# print("Accuracy of: ", np.mean(lasso_predicted.astype(int) == y_test_onehot.astype(int)))

[0.11492712 0.11492712 0.11492712 ... 0.11492712 0.11492712 0.11492712]


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous targets

https://stackoverflow.com/questions/54589669/confusion-matrix-error-classification-metrics-cant-handle-a-mix-of-multilabel

In [43]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)

# clf.fit(X_train,y_train)

cross_val_score(model, X

lr_predicted = clf.predict(X_test)

print(clf.intercept_) #
print(clf.coef_) #feature significance

print("Accuracy of Logistic Regression:", np.mean(lr_predicted.astype(int) == y_test.astype(int)))

SyntaxError: invalid syntax (<ipython-input-43-fb27a6d5d8bc>, line 9)

# VERY IMPORTANT OBSERVATION: CHANGING THE BINING (i.e. setting value < 3 to return 1) changes the accuracy significantly

### Solution: ONE HOT ENCODING https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
Our computer now knows how to represent these categories, because it knows how to work with numbers. However, this method of encoding is not very effective, because it tends to naturally give the higher numbers higher weights.

Next, we can create a binary vector to represent each integer value. The vector will have a length of 2 for the 2 possible integer values.


https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/

The ‘red’ label encoded as a 0 will be represented with a binary vector [1, 0] where the zeroth index is marked with a value of 1. In turn, the ‘green’ label encoded as a 1 will be represented with a binary vector [0, 1] where the first index is marked with a value of 1.

In [27]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)


['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']
